In [ ]:
import sys; sys.path.append('../../utils')
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set_theme(style='darkgrid')

from utils import save_plot

In [ ]:
MONK_TASK = 1

In [ ]:
TRAIN_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.train')
TEST_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.test')
IMAGES_FOLDER = os.path.join('..', '..', '..', 'images', 'monk', f'task-{MONK_TASK}', 'neural_network')
MODEL_FOLDER = os.path.join('..', '..', '..', 'trained_models', 'monk', f'task-{MONK_TASK}')

In [ ]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 8))

df_train = pd.read_csv(TRAIN_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_test = pd.read_csv(TEST_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_train.head()

In [ ]:
features = ['feature_' + str(i) for i in range(1, 7)]

# Rename columns
new_column_names = ['class'] + features

df_train.columns = new_column_names
df_test.columns = new_column_names

df_train.head()

In [ ]:
features_to_encode = [col for col in features if df_train[col].nunique() > 2]

df_train_encoded = pd.get_dummies(df_train, columns=features_to_encode)
df_test_encoded = pd.get_dummies(df_test, columns=features_to_encode)

df_train_encoded, df_test_encoded = df_train_encoded.align(df_test_encoded, join='inner', axis=1)

df_train_encoded.head()

In [ ]:
features = df_train_encoded.columns.difference(['class'])

X_train = df_train_encoded[features].to_numpy()
y_train = df_train_encoded['class'].to_numpy()

X_test = df_test_encoded[features].to_numpy()
y_test = df_test_encoded['class'].to_numpy()

# Create model

In [ ]:
from keras.src.optimizers import Adam, SGD
from keras.models import Sequential
from keras.layers import Dense

def create_model(architecture=(8,), optimizer='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Dense(units=architecture[0], input_dim=X_train.shape[1], activation='relu'))

    for units in architecture[1:]:
        model.add(Dense(units=units, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate)

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

model = KerasClassifier(create_model, verbose=0)

pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('neuralnetwork', model)
])

In [ ]:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'neuralnetwork__model__architecture': [
        (64,), (64, 32), (128,), (128, 64), (128, 64, 32)
    ],
    'neuralnetwork__model__optimizer': ['adam', 'sgd'],
    'neuralnetwork__model__learning_rate': loguniform(1e-4, 0.1),
    'neuralnetwork__epochs': range(10, 100, 10),
    'neuralnetwork__batch_size': range(32, 128, 32)
}

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=100,
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

final_model = random_search.best_estimator_
y_pred = final_model.predict(X_test)

print('Best parameters: ', random_search.best_params_)
print('Best accuracy: ', random_search.best_score_)
print('Test set accuracy: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Learning curve

In [ ]:
best_keras_model = final_model.named_steps['neuralnetwork'].model

history = best_keras_model.history

In [ ]:
pd.DataFrame(history.history).plot(
    figsize=(8, 5),
    xlim=[0, 29],
    ylim=[0, 1],
    grid=True,
    xlabel='Epoch',
    style=['r--', 'r--.', 'b-', 'b-*']
)
plt.show()

# Save model

In [ ]:
from joblib import dump

model_path = os.path.join(MODEL_FOLDER, 'NN_model.joblib')
dump(final_model, model_path, compress=3)